### P4_Section 2 - *Processing Scrapped Data*

In [217]:
import pandas as pd
import numpy as np
import string
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import warnings
warnings.simplefilter(action='ignore', category=RuntimeWarning)
from sklearn import metrics
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from textblob import TextBlob, Word
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB

In [70]:
cd /Users/sherf/ 

/Users/sherf


### Section 2 - Part 1 - Data Preprocessing

- In this section we load the scrapped data and perform simple EDA as the following:

1- Add column names

2- Sort raws descending based on *comapny name*

3- Drop duplicate values

4- In *Salary* column we replace all the nan with nothing found


In [244]:
df=pd.read_csv("./Downloads/indeed_analyst_k.csv")
columns = ['Index','job_title', 'salary','company_name', 'location','summary']
df.columns=columns
df.sort_values(['company_name'],axis=0,ascending=False,inplace=True)
jobs_list=df.drop_duplicates(subset=['job_title','location'])
jobs_list.head()

,Index,job_title,salary,company_name,location,summary
4515,4515,LEGEND SEM | PPC Specialist,$30 - $60 an hour,wildminds agency,Windsor VIC,*Like to push the boundaries with digital mark...
3289,3289,ACCOUNT MANAGERS,NaN,uberbrand,Sydney NSW,About this role You often read about roles bei...
1422,1422,Business Analyst Power BI,NaN,u&u Recruitment Partners,Sydney Western Suburbs NSW,Use your business analysis and technical BI sk...
2272,2272,iOS QA Engineer,NaN,real time,Adelaide SA,About the company:Real Time Data (RTD) is a pi...
1484,1484,Litigation Support Analyst,NaN,people2people,Sydney NSW,THE FIRM This global firm is a renowned law fi...


In [72]:
jobs_list['salary']=jobs_list['salary'].apply(lambda x:str(x).replace('nan','Nothing found'))
jobs_list=jobs_list.reset_index(drop=True)
jobs_list.summary.iloc[0]

/Users/sherf/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [211]:
# jobs_list['salary'].apply(pd.Series.value_counts).nunique()
# jobs_list.sort_values(['salary'],axis=0,ascending=False).nunique()

Using string library to remove all unwanted special characters

In [78]:
st=string.punctuation
st

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

### In the section below I have replaced the punctuation on two steps:

1- Manually to control the replacement of characters

2- Automatic to remove the rest of unwanted characters

In [79]:
%%time 

columns=jobs_list.columns
for i in columns:
    jobs_list[i]=jobs_list[i].apply(lambda x:str(x).replace(',',''))
    jobs_list[i]=jobs_list[i].apply(lambda x:str(x).replace('$',' '))
    jobs_list[i]=jobs_list[i].apply(lambda x:str(x).replace('-',' '))
    jobs_list[i]=jobs_list[i].apply(lambda x:str(x).replace('_',' '))
    jobs_list[i]=jobs_list[i].apply(lambda x:str(x).replace(';',' '))
    jobs_list[i]=jobs_list[i].apply(lambda x:str(x).replace('|',' '))
    jobs_list[i]=jobs_list[i].apply(lambda x:str(x).replace('&',' and '))
    for j in st:
        jobs_list[i]=jobs_list[i].apply(lambda x:str(x).replace(j,''))

CPU times: user 525 ms, sys: 21.7 ms, total: 547 ms
Wall time: 557 ms


In [80]:
jobs_list.summary.iloc[0]

'Like to push the boundaries with digital marketing and SEM campaignswildminds is a creative ideas first agency We’re a bit smarter  bit faster  more cunning than most We work with businesses to get them out of the digital wilderness  with killer ideas and marketing servicesWe’re a small team of great minds who don’t mind life on the quirky side We like to push boundaries  encourage ideas and input  we work as a team but also operate solo to get the job done We’re looking for a epic SEM guru who knows their way around Adwords  InDesign  and all things digital marketingTo be successful in this role you will have demonstrable experience driving exceptional outcomes through Google AdWords  Google Tag Manager GTM  Google Display Network  YouTube and Paid Social You will also be well versed with Google Analytics with the ability to use data driven marketing to make key decisions on clients websites in terms of improving sales and lead generationDo you have what we wantA minimum of 2 3 years

In [82]:
#Extracting jobs without salary
salary_list=jobs_list.drop(jobs_list[jobs_list['salary'] !='Nothing found'].index)
len(salary_list)

1712

- In the section below I have tackled all the different salary types and converted all to annual salary

In [83]:
#finding salary catagories
hourly_rate=jobs_list[jobs_list['salary'].str.contains("hour")==True]
daily_rate=jobs_list[jobs_list['salary'].str.contains("day")==True]
weekly_rate=jobs_list[jobs_list['salary'].str.contains("week")==True]
yearly_rate=jobs_list[jobs_list['salary'].str.contains("year")==True]

In [84]:
def salary_avg(salary_rate):
    salary_avg=[]
    for i in range(0,(len(salary_rate))):
        b=salary_rate['salary'].iloc[i]
        a=[int(s) for s in b.split() if s.isdigit()]
        avg_salary=np.mean(a)
        salary_avg.append(avg_salary)
    return salary_avg

In [85]:
hourly_rate['avg_annual_salary']=salary_avg(hourly_rate)
hourly_rate['avg_annual_salary']=hourly_rate['avg_annual_salary'].apply(lambda x:np.multiply(x,2080))

daily_rate['avg_annual_salary']=salary_avg(daily_rate)
daily_rate['avg_annual_salary']=daily_rate['avg_annual_salary'].apply(lambda x:np.multiply(x,260))

weekly_rate['avg_annual_salary']=salary_avg(weekly_rate)
weekly_rate['avg_annual_salary']=weekly_rate['avg_annual_salary'].apply(lambda x:np.multiply(x,52))

yearly_rate['avg_annual_salary']=salary_avg(yearly_rate)

/Users/sherf/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/sherf/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/sherf/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docum

- Create a dataframe consist of all jobs with salary

In [86]:
jobs_with_salary=hourly_rate.append(daily_rate)
jobs_with_salary=jobs_with_salary.append(weekly_rate)
jobs_with_salary=jobs_with_salary.append(yearly_rate)

In [87]:
print('Min Salary =',jobs_with_salary.avg_annual_salary.min())
print('Max Salary =',jobs_with_salary.avg_annual_salary.max())

Min Salary = 17420.0
Max Salary = 8987680.0


### Section 2 - Part 2 - Understanding Factors that impact salary

To better understand salary, We divide it into three *BALANCED* classes: 

Cat 0 - *less Than 85k* 

Cat 1 - *Between 85k and 120k*

Cat 2 - *More than 120k*

In [88]:
Cat_0=jobs_with_salary[(jobs_with_salary.avg_annual_salary.values <=85000)]
Cat_0['salary_catagory']=0
                       
Cat_1=jobs_with_salary[(jobs_with_salary.avg_annual_salary>85000) & (jobs_with_salary.avg_annual_salary<=120000)]
Cat_1['salary_catagory']=1
                                                                                                      
Cat_2=jobs_with_salary[(jobs_with_salary.avg_annual_salary.values >120000)]
Cat_2['salary_catagory']=2

/Users/sherf/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/sherf/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/Users/sherf/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-do

In [212]:
Cat_2.head()

,Index,job_title,salary,company_name,location,summary,avg_annual_salary,salary_catagory
109,662,Clinical Trials Nurse,3158 4434 an hour,Western Sydney Local Health District,Sydney Western Suburbs NSW,Employment Type Temporary Full Time Up to 12 m...,7895680.0,2
347,1394,Database Business Analyst,70 90 an hour,Talent International,Canberra ACT,Oracle Database SQL Server 12 month contract ...,166400.0,2
831,222,Medical Laboratory Scientist,62 an hour,NSW Health Pathology,Sydney NSW,Employment Type Temporary Full Time Position C...,128960.0,2
949,1556,FP A Manager Global Retail 6 Month Contract,70 80 an hour,Michael Page,Sydney NSW,6 month contract with a leading cosmetics bran...,156000.0,2
1241,1401,Data Analyst,110 115 an hour,Hudson,Canberra ACT,12 month contract role with possible extension...,234000.0,2


In [89]:
# Joining salary classes to form single df
jobs_with_salary_cat=Cat_0.append(Cat_1).append(Cat_2)

In [213]:
# checking to make sure we have a balance in the different classes
jobs_with_salary_cat.salary_catagory.value_counts(normalize=True)

1    0.348790
2    0.336694
0    0.314516
Name: salary_catagory, dtype: float64

In [91]:
jobs_with_salary_cat.shape

(496, 8)

In [214]:
#droppind index column and resetting Index
jobs_with_salary_cat.drop(axis=1,columns='Index',inplace=True)
jobs_with_salary_cat=jobs_with_salary_cat.reset_index(drop=True)
jobs_with_salary_cat.head()

,job_title,salary,company_name,location,summary,avg_annual_salary,salary_catagory
0,Demand Planner,38 40 an hour,people2people,Sydney NSW,The company is a global leader in technology ...,81120.0,0
1,Marketing Angel,25 an hour,Impact HR,Northern Beaches NSW,Who is Impact HRPassionate about business’ peo...,52000.0,0
2,Lodgement Delivery Officers Wanted,30 40 an hour,Hudson,Brisbane QLD,Join a Leading Federal Government Department ...,72800.0,0
3,Project Administration,28 an hour,HAYS,Melbourne City Centre VIC,5 6 week temp assignment travelling to variou...,58240.0,0
4,Admin Assistant,29 an hour,HAYS,Melbourne City Centre VIC,Temp to perm admin job available in Melbourne ...,60320.0,0


In [311]:
# print the first review
print(jobs_with_salary_cat.summary[0])

 Like to push the boundaries with digital marketing and SEM campaigns wildminds is a creative ideas first agency  We’re a bit smarter  bit faster  more cunning than most  We work with businesses to get them out of the digital wilderness  with killer ideas and marketing services We’re a small team of great minds who don’t mind life on the quirky side  We like to push boundaries  encourage ideas and input  we work as a team but also operate solo to get the job done  We’re looking for a epic SEM guru who knows their way around Adwords  InDesign  and all things digital marketing To be successful in this role you will have demonstrable experience driving exceptional outcomes through Google AdWords  Google Tag Manager GTM  Google Display Network  YouTube and Paid Social  You will also be well versed with Google Analytics with the ability to use data driven marketing to make key decisions on clients  websites in terms of improving sales and lead generation Do you have what we want A minimum o

### Text Blob & Lemmatiztion

In [215]:
# This function creates a text blob and lemmatize text

def split_into_lemmas(text):
    text = str(text).lower()
    words = TextBlob(text).words
    return [word.lemmatize() for word in words]


### Section 2 - Part 3 - Building a classification Model to classify Salary Class based on Summary text

- In this section we will build a classifcation model to classify salary based on job summary

- First we will use Naive Bayes Classifier with TfidfVectorizer and CountVectorizer to comapre performance

- Second we will feed our data Sequentially to a pipline to try a list of transforms and estimators

### Section 2 - Part 3.1 - classification Models 

In [220]:
#Instantiate TfidfVectorizer
tf = TfidfVectorizer(analyzer=split_into_lemmas)

# Creating the Bag of Words model
X = tf.fit_transform(jobs_with_salary_cat.summary).toarray()
y = jobs_with_salary_cat.iloc[:,6].values

# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2 )

# Fitting Naive Bayes to the Training set
classifier = GaussianNB()
classifier.fit(X_train, y_train)

# Predicting the Test set results
y_pred = classifier.predict(X_test)

#Model Validation
scores=cross_val_score(classifier,X,y,cv=5)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

print('Accuracy: ', metrics.balanced_accuracy_score(y_test, y_pred))
print('Accuracy: ', metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.37 (+/- 0.04)
Accuracy:  0.44953742640874683
Accuracy:  0.44


In [221]:
#Instantiate CountVectorizer
cv = CountVectorizer(analyzer=split_into_lemmas)

# Creating the Bag of Words model
X = cv.fit_transform(jobs_with_salary_cat.summary).toarray()
y = jobs_with_salary_cat.iloc[:,6].values

# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2 )

# Fitting Naive Bayes to the Training set
classifier = GaussianNB()
classifier.fit(X_train, y_train)

# Predicting the Test set results
y_pred = classifier.predict(X_test)

#Model Validation
scores=cross_val_score(classifier,X,y,cv=5)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

print('Accuracy: ', metrics.balanced_accuracy_score(y_test, y_pred))
print('Accuracy: ', metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.38 (+/- 0.07)
Accuracy:  0.5710041592394534
Accuracy:  0.57


**It seems that CountVectorizer is giving us better results than TfidfVectorizer**

### Testing model on unseen document

In [240]:
X_new=cv.transform(jobs_list.summary).toarray()

y_pred2 = classifier.predict(X_new)

y_pred2

array([1, 1, 2, ..., 2, 2, 1])

In [241]:
jobs_list['Model Generalization Results']=y_pred2
jobs_list.head()

,Index,job_title,salary,company_name,location,summary,Model Generalization Results
0,4515,LEGEND SEM PPC Specialist,30 60 an hour,wildminds agency,Windsor VIC,Like to push the boundaries with digital marke...,1
1,3289,ACCOUNT MANAGERS,Nothing found,uberbrand,Sydney NSW,About this role You often read about roles bei...,1
2,1422,Business Analyst Power BI,Nothing found,u and u Recruitment Partners,Sydney Western Suburbs NSW,Use your business analysis and technical BI sk...,2
3,2272,iOS QA Engineer,Nothing found,real time,Adelaide SA,About the companyReal Time Data RTD is a pione...,0
4,1484,Litigation Support Analyst,Nothing found,people2people,Sydney NSW,THE FIRM This global firm is a renowned law fi...,1


### Section 2 - Part 3.2 - Feeding the data to a Pipeline to compare different models

**1- Stochastic Gradient Descent (SGD)** 

SGD  applied to large-scale and sparse machine learning problems often encountered in text classification and natural language processing. Given that the data is sparse, the classifiers in this module easily scale to problems with more than 10^5 training examples and more than 10^5 features.

**The advantages of Stochastic Gradient Descent are:**
 - Efficiency.
 - Ease of implementation (lots of opportunities for code tuning).

**The disadvantages of Stochastic Gradient Descent include:**
 - SGD requires a number of hyperparameters such as the regularization parameter and the number of iterations.
 - SGD is sensitive to feature scaling.

**2- Logistic Regression** 

In the multiclass case, the training algorithm uses the one-vs-rest (OvR) scheme if the ‘multi_class’ option is set to ‘ovr’, and uses the cross-entropy loss if the ‘multi_class’ option is set to ‘multinomial’.

The ‘liblinear’ solver supports both L1 and L2 regularization, with a dual formulation only for the L2 penalty. The Elastic-Net regularization is only supported by the ‘saga’ solver.


In [224]:
X=jobs_with_salary_cat.summary
# y=jobs_with_salary_cat.iloc[:,6].values

In [231]:
# Define a pipeline combining a text feature extractor with a simple
# classifier
from pprint import pprint
from time import time
import logging

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier, LogisticRegressionCV
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', SGDClassifier(tol=1e-3)),
])

pipeline2 = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf2', LogisticRegressionCV(cv=5, random_state=0,multi_class='multinomial')),
])
# uncommenting more parameters will give better exploring power but will
# increase processing time in a combinatorial way
parameters = {
    'vect__max_df': (0.5, 0.75, 1.0),
    # 'vect__max_features': (None, 5000, 10000, 50000),
    'vect__ngram_range': ((1, 1), (1, 2),(1,3)),  # unigrams or bigrams
    # 'tfidf__use_idf': (True, False),
    # 'tfidf__norm': ('l1', 'l2'),
    'clf__max_iter': (20,),
    'clf__alpha': (0.00001, 0.000001),
    'clf__penalty': ('l2', 'elasticnet'),
    # 'clf__max_iter': (10, 50, 80),
}


parameters2 = {
    'vect__max_df': (0.5, 0.75, 1.0),
    # 'vect__max_features': (None, 5000, 10000, 50000),
    'vect__ngram_range': ((1, 1), (1, 2),(1,3)),  # unigrams or bigrams
    # 'tfidf__use_idf': (True, False),
    # 'tfidf__norm': ('l1', 'l2'),
    'clf2__max_iter': (20,),
#     'clf2__alpha': (0.00001, 0.000001),
    #'clf2__penalty': (l2),
    # 'clf__max_iter': (10, 50, 80),
}

# multiprocessing requires the fork to happen in a __main__ protected
# block

# find the best parameters for both the feature extraction and the
# classifier
grid_search1 = GridSearchCV(pipeline, parameters, cv=5,
                       n_jobs=4, verbose=1)

print("Performing grid search...")
print("pipeline:", [name for name, _ in pipeline.steps])
print("parameters:")
pprint(parameters)
t0 = time()
grid_search1.fit(X.tolist(), y)
print("done in %0.3fs" % (time() - t0))
print()

print("Best score: %0.3f" % grid_search1.best_score_)
print("Best parameters set:")
best_parameters = grid_search1.best_estimator_.get_params()

for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))
    
print('________________________________________________________')

grid_search2 = GridSearchCV(pipeline2, parameters2, cv=5,
                       n_jobs=4, verbose=1)

print("Performing grid search...")
print("pipeline:", [name for name, _ in pipeline2.steps])
print("parameters:")
pprint(parameters2)
t0 = time()
grid_search.fit(X.tolist(), y)
print("done in %0.3fs" % (time() - t0))
print()

print("Best score: %0.3f" % grid_search.best_score_)
print("Best parameters set:")
best_parameters = grid_search.best_estimator_.get_params()

for param_name in sorted(parameters2.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

Performing grid search...
pipeline: ['vect', 'tfidf', 'clf']
parameters:
{'clf__alpha': (1e-05, 1e-06),
 'clf__max_iter': (20,),
 'clf__penalty': ('l2', 'elasticnet'),
 'vect__max_df': (0.5, 0.75, 1.0),
 'vect__ngram_range': ((1, 1), (1, 2), (1, 3))}
Fitting 5 folds for each of 36 candidates, totalling 180 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:   21.9s
[Parallel(n_jobs=4)]: Done 180 out of 180 | elapsed:  1.5min finished
/Users/sherf/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/Users/sherf/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:603: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.


done in 93.156s

Best score: 0.484
Best parameters set:
	clf__alpha: 1e-06
	clf__max_iter: 20
	clf__penalty: 'l2'
	vect__max_df: 1.0
	vect__ngram_range: (1, 3)
________________________________________________________
Performing grid search...
pipeline: ['vect', 'tfidf', 'clf2']
parameters:
{'clf2__max_iter': (20,),
 'vect__max_df': (0.5, 0.75, 1.0),
 'vect__ngram_range': ((1, 1), (1, 2), (1, 3))}
Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=4)]: Done  45 out of  45 | elapsed: 19.2min finished
/Users/sherf/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/Users/sherf/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/Users/sherf/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/Users/sherf/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", Convergenc

done in 1154.717s

Best score: 0.494
Best parameters set:
	clf2__max_iter: 20
	vect__max_df: 0.75
	vect__ngram_range: (1, 1)


### Section 3 - Job Catagory Factors - Topic Modeling using `Latent Dirichlet Allocation (LDA) 

In this part, I will try to find the factors that distinguish job category using LDA model to map all the documents to the topics that best describe the job catagories.

**Section 3 - Part 1 - Prepropcessing**

*Step 1: Convert documents to a single list/matrix.*

*Step 2: Tokenization to break text into tokens using the help og regualer expressions.*

*Step 3: Stemming the words into roots*

*Step 4: Stop words removal*

*Step 5: Apply to all documents*

In [122]:
from gensim import corpora, models, matutils
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from collections import defaultdict
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import RegexpTokenizer
import gensim

In [173]:
titles=jobs_list['job_title'].values
titles=titles.tolist()

In [148]:
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
stop_words = list(ENGLISH_STOP_WORDS)
tokenizer = RegexpTokenizer('^\D+')
p_stemmer = PorterStemmer()

In [149]:
t1=titles[0]
tokenizer.tokenize(t1)

['LEGEND SEM   PPC Specialist']

In [150]:
# list for tokenized documents in loop
texts = []

# loop through document list
for i in titles:
    
    # clean and tokenize document string
    raw = i.lower()
    tokens = tokenizer.tokenize(raw)

    # remove stop words from tokens
    stopped_tokens = [i for i in tokens if not i in stop_words]
    
    # stem tokens
    stemmed_tokens = [p_stemmer.stem(i) for i in stopped_tokens]
    
    # add tokens to list
    texts.append(stemmed_tokens)

**Section 3 - Part 2 - Running LDA**

In [161]:
# turn our tokenized documents into a id <-> term dictionary
dictionary = corpora.Dictionary(texts)
    
# convert tokenized documents into a document-term matrix
corpus = [dictionary.doc2bow(text) for text in texts]

# generate LDA model
ldamodel = gensim.models.LdaModel(corpus, num_topics=4, id2word = dictionary, passes=20)

In [152]:
print(ldamodel.print_topics(num_topics=2, num_words=4))

[(1, '0.006*"marketing analyst" + 0.006*"senior security analyst" + 0.005*"senior data engin" + 0.005*"process engin"'), (2, '0.009*"cyber security analyst" + 0.008*"senior business analyst" + 0.007*"electronics engin" + 0.007*"data scientist   educ"')]


In [162]:
print(ldamodel.print_topics(num_topics=2, num_words=2))

[(2, '0.009*"cyber security analyst" + 0.009*"information security analyst"'), (0, '0.009*"online analyst – saba  sportscraft   jag" + 0.006*"bi develop"')]


***Trying different LDA Models to find a better results***

In [163]:
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *

In [164]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /Users/sherf/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [232]:
# stemmer = SnowballStemmer('english')
# original_words = ['caresses', 'flies', 'dies', 'mules', 'denied','died', 'agreed', 'owned', 
#            'humbled', 'sized','meeting', 'stating', 'siezing', 'itemization','sensational', 
#            'traditional', 'reference', 'colonizer','plotted']
# singles = [stemmer.stem(plural) for plural in original_words]
# pd.DataFrame(data = {'original word': original_words, 'stemmed': singles})

In [178]:
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [177]:
doc_sample = titles[101]

print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))

original document: 
['Associate', 'Relationship', 'Manager', '', 'Infrastructure', '', 'Energy', 'and', 'Resources', '', 'CIB']


 tokenized and lemmatized document: 
['associ', 'relationship', 'manag', 'infrastructur', 'energi', 'resourc', 'cib']


In [197]:
documents=jobs_list['summary']
processed_docs = documents.map(preprocess)

In [198]:
dictionary = gensim.corpora.Dictionary(processed_docs)

In [199]:
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 abil
1 adword
2 agenc
3 analysi
4 analyt
5 bid
6 bing
7 bonus
8 boundari
9 budget
10 busi


In [200]:
dictionary.filter_extremes(no_below=10, no_above=0.5, keep_n=100000)

In [201]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

In [233]:
# bow_doc_4310 = bow_corpus[100]

# for i in range(len(bow_doc_4310)):
#     print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0], 
#                                                      dictionary[bow_doc_4310[i][0]], 
#                                                      bow_doc_4310[i][1]))

In [234]:
# tfidf = models.TfidfModel(bow_corpus)
# corpus_tfidf = tfidf[bow_corpus]
# from pprint import pprint

# for doc in corpus_tfidf:
#     pprint(doc)
#     break

In [208]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=20, workers=2)

In [206]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.014*"custom" + 0.011*"research" + 0.009*"client" + 0.009*"market" + 0.009*"design" + 0.008*"product" + 0.006*"analysi" + 0.006*"report" + 0.005*"secur" + 0.005*"australia"
Topic: 1 
Words: 0.012*"market" + 0.007*"client" + 0.007*"report" + 0.006*"inform" + 0.006*"secur" + 0.006*"perform" + 0.006*"time" + 0.005*"risk" + 0.005*"analyst" + 0.005*"oper"
Topic: 2 
Words: 0.011*"custom" + 0.007*"client" + 0.006*"report" + 0.006*"peopl" + 0.006*"compani" + 0.005*"intellig" + 0.005*"industri" + 0.005*"australia" + 0.005*"thale" + 0.005*"analysi"
Topic: 3 
Words: 0.011*"client" + 0.010*"custom" + 0.007*"market" + 0.006*"compani" + 0.006*"learn" + 0.006*"candid" + 0.006*"posit" + 0.006*"research" + 0.005*"employ" + 0.005*"technolog"
Topic: 4 
Words: 0.011*"technolog" + 0.010*"custom" + 0.009*"client" + 0.009*"product" + 0.007*"drive" + 0.007*"technic" + 0.007*"market" + 0.006*"consult" + 0.006*"build" + 0.006*"engin"
Topic: 5 
Words: 0.011*"report" + 0.010*"secur" + 0.009*"inf

In [195]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=2, id2word=dictionary, passes=2, workers=4)
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.072*"analyst" + 0.063*"data" + 0.049*"scientist" + 0.039*"analyt" + 0.035*"senior" + 0.030*"manag" + 0.028*"busi" + 0.027*"research" + 0.025*"consult" + 0.023*"secur"
Topic: 1 Word: 0.060*"engin" + 0.055*"analyst" + 0.047*"develop" + 0.042*"busi" + 0.041*"manag" + 0.036*"senior" + 0.035*"data" + 0.031*"intellig" + 0.027*"consult" + 0.022*"market"


In [209]:
for index, score in sorted(lda_model[bow_corpus[3]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))



Score: 0.44329068064689636	 
Topic: 0.020*"technolog" + 0.014*"product" + 0.013*"custom" + 0.011*"technic" + 0.011*"learn" + 0.011*"engin" + 0.010*"softwar" + 0.010*"consult" + 0.009*"client" + 0.009*"design"

Score: 0.3367806077003479	 
Topic: 0.018*"engin" + 0.012*"oper" + 0.010*"technic" + 0.010*"system" + 0.009*"product" + 0.009*"industri" + 0.007*"time" + 0.007*"design" + 0.007*"ensur" + 0.006*"equip"

Score: 0.2168297916650772	 
Topic: 0.021*"report" + 0.013*"analysi" + 0.011*"stakehold" + 0.011*"analyst" + 0.009*"insight" + 0.009*"custom" + 0.008*"intellig" + 0.008*"excel" + 0.008*"perform" + 0.007*"deliv"


In [242]:
# topics_labels = {
#    0: "Data Analyst",
#    1: "Data Scientest",
#    2: "Security Analyst"
# }

# doc_topics = [lda.get_document_topics(doc) for doc in corpus]

# topic_data = []

# for document_id, topics in enumerate(doc_topics):
    
#     document_topics = []
    
#     for topic, probability in topics:
       
#         topic_data.append({
#             'document_id':  document_id,
#             'topic_id':     topic,
#             'topic':        topics_labels[topic],
#             'probability':  probability
#         })

# topics_df = pd.DataFrame(topic_data)
# topics_df.pivot_table(values="probability", index=["document_id", "topic"])